In [1]:
from platform import python_version

In [2]:
python_version()

'3.8.6'

In [3]:
!pip install psycopg2-binary
!pip install pandas

Active code page: 1252
Active code page: 1252


In [4]:
import psycopg2
import pandas as pd

In [5]:
def create_database():
    # Create a connection to Postgres DB
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=243278") 
    conn.set_session(autocommit=True)
    cur =conn.cursor()

    # Create database 
    cur.execute("DROP DATABASE IF EXISTS movies;")
    cur.execute("CREATE DATABASE movies;")

    conn.close()
    
    #Connect to movies db
    conn = psycopg2.connect("host=127.0.0.1 dbname=movies user=postgres password=243278")
    cur =conn.cursor()
    conn.set_session(autocommit=True)
    
    return cur,conn

In [6]:
def create_table(cur,conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [7]:
def drop_table(cur,conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [8]:
moviesData = pd.read_csv("ml-latest-small/movies.csv")

In [9]:
moviesData.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
#moviesData_clean = moviesData[["movieId","title","genres"]]

In [11]:
#moviesData_clean.head()

In [12]:
ratingsData = pd.read_csv("ml-latest-small/ratings.csv")

In [13]:
ratingsData.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [14]:
#ratingsData.columns

In [15]:
#ratingsData = ratingsData.drop(["timestamp"], axis=1)

In [16]:
tagsData = pd.read_csv("ml-latest-small/tags.csv")

In [17]:
tagsData.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [18]:
cur,conn = create_database()

In [19]:
movies_data_table_create = ("""CREATE TABLE IF NOT EXISTS moviesdata(
movies_id int PRIMARY KEY,
title varchar,
genres varchar
)""")

In [20]:
cur.execute(movies_data_table_create)
conn.commit()

In [21]:
ratings_data_table_create = ("""CREATE TABLE IF NOT EXISTS ratingsdata(
user_id int,
movies_id int,
rating float,
timestamp varchar,
FOREIGN KEY (movies_id) REFERENCES moviesdata(movies_id)
)""")

In [22]:
cur.execute(ratings_data_table_create)
conn.commit()

In [25]:
tags_data_table_create = ("""CREATE TABLE IF NOT EXISTS tagsdata(
user_id int,
movies_id int,
tags varchar,
timestamp varchar,
FOREIGN KEY (movies_id) REFERENCES moviesdata(movies_id)
)""")

In [26]:
cur.execute(tags_data_table_create)
conn.commit()

In [27]:
movies_data_table_insert = ("""INSERT INTO moviesdata(
movies_id,
title,
genres)
VALUES(%s,%s,%s)
""")

In [28]:
for i, row in moviesData.iterrows():
    #print(row)
    cur.execute(movies_data_table_insert, list(row))

In [29]:
ratings_data_table_insert = ("""INSERT INTO ratingsdata(
user_id,
movies_id,
rating,
timestamp)
VALUES(%s,%s,%s,%s)
""")

In [30]:
for i, row in ratingsData.iterrows():
    #print(row)
    cur.execute(ratings_data_table_insert, list(row))

In [33]:
tags_data_table_insert = ("""INSERT INTO tagsdata(
user_id,
movies_id,
tags,
timestamp)
VALUES(%s,%s,%s,%s)
""")

In [34]:
for i, row in tagsData.iterrows():
    #print(row)
    cur.execute(tags_data_table_insert, list(row))

In [42]:
cur.execute("""SELECT t1.movies_id,t1.title
FROM moviesdata t1
INNER JOIN ratingsdata t2
ON t1.movies_id = t2.movies_id
WHERE t2.user_id = 2""")

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

#Print movies that user_id= 2 rated

(318, 'Shawshank Redemption, The (1994)')
(333, 'Tommy Boy (1995)')
(1704, 'Good Will Hunting (1997)')
(3578, 'Gladiator (2000)')
(6874, 'Kill Bill: Vol. 1 (2003)')
(8798, 'Collateral (2004)')
(46970, 'Talladega Nights: The Ballad of Ricky Bobby (2006)')
(48516, 'Departed, The (2006)')
(58559, 'Dark Knight, The (2008)')
(60756, 'Step Brothers (2008)')
(68157, 'Inglourious Basterds (2009)')
(71535, 'Zombieland (2009)')
(74458, 'Shutter Island (2010)')
(77455, 'Exit Through the Gift Shop (2010)')
(79132, 'Inception (2010)')
(80489, 'Town, The (2010)')
(80906, 'Inside Job (2010)')
(86345, 'Louis C.K.: Hilarious (2010)')
(89774, 'Warrior (2011)')
(91529, 'Dark Knight Rises, The (2012)')
(91658, 'Girl with the Dragon Tattoo, The (2011)')
(99114, 'Django Unchained (2012)')
(106782, 'Wolf of Wall Street, The (2013)')
(109487, 'Interstellar (2014)')
(112552, 'Whiplash (2014)')
(114060, 'The Drop (2014)')
(115713, 'Ex Machina (2015)')
(122882, 'Mad Max: Fury Road (2015)')
(131724, 'The Jinx: Th